<h2>FINAL DATA PREPROCESSING & MODEL TRAINING</h2>

Script ini digunakan untuk mempreprocess data menjadi data final yang dapat digunakan untuk training model, dan training model LSTM itu sendiri. Secara umum, script ini meliputi:
<h4>Data Preprocessing</h4>
- Padding & Truncating data, pada dasarnya memastikan seluruh trj_id memiliki jumlah data yang sama, yakni 20 data, dengan menambahkan data yang kurang dengan 0 dan memotong data yang kelebihan dan mengambil 20 data paling belakang.
- Membagi data menjadi X (koordinat, keceptan, dll yang dimasukkan ke model untuk melatih model) dan y (koordinat yang benar untuk dibandingkan dengan koordinat hasil prediksi model). y akan diambil dari koordinat terakhir tiap trajectory.
- Membagi data menjadi training data dan testing data, dengan pembagian 90%/10%.
<br/>
<h4>Model Training</h4>
- Mendefinisikan dan melatih model LSTM berdasarkan data yang telah siap.
- Arsitektur, epoch, loss, optimizer, dan dll dapat dengan bebas diubah-ubah untuk mencoba model lain.
<br/>
Jika ingin melihat performa model pertama yang telah saya latih, maka model itu sudah disave dalam folder models dengan nama model_v1.keras. Model dapat ditest menggunakan script Model Test.ipynb

In [1]:
## Import libraries
import numpy as np
import tensorflow as tf
import datetime
import keras

In [2]:
# Load Tensorboard
%load_ext tensorboard

In [3]:
## Load numpy test data
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
simple_data = np.load('simple_data.npy')
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')

In [4]:
## Define normalization layer
normalize_layer = tf.keras.layers.Normalization(axis=-1)
denormalize_layer = tf.keras.layers.Normalization(axis=-1, invert=True)

normalize_layer.adapt(simple_data)
denormalize_layer.adapt(simple_data[:, :, 0:2])

normalize_weights = normalize_layer.get_weights()
mean_variance = np.array([normalize_weights[0][0:2], normalize_weights[1][0:2]])

In [5]:
## Define custom metric(s)
@keras.saving.register_keras_serializable(package="custom_metrics", name="avg_m_diff")
def avg_m_diff(y_true, y_pred):
    # Ensure the input tensors have the correct shape
    assert y_true.shape == y_pred.shape
    assert y_true.shape[2] == 2
    
    def haversine_distance(lat1, lon1, lat2, lon2):
        # Radius of the Earth in meters
        R = 6371000.0
        
        # Convert latitude and longitude from degrees to radians
        lat1 = lat1 * (tf.constant(3.141592653589793) / 180.0)
        lon1 = lon1 * (tf.constant(3.141592653589793) / 180.0)
        lat2 = lat2 * (tf.constant(3.141592653589793) / 180.0)
        lon2 = lon2 * (tf.constant(3.141592653589793) / 180.0)
        
        # Compute differences
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        # Haversine formula
        a = tf.math.sin(dlat / 2)**2 + tf.math.cos(lat1) * tf.math.cos(lat2) * tf.math.sin(dlon / 2)**2
        c = 2 * tf.math.atan2(tf.math.sqrt(a), tf.math.sqrt(1 - a))
        
        # Distance in meters
        distance = R * c
        
        return distance
    
    # Reshape the tensors to 2D arrays for easier manipulation
    y_true_flat = tf.reshape(y_true, [-1, 2])
    y_pred_flat = tf.reshape(y_pred, [-1, 2])
    
    # Split the coordinates into separate tensors
    lat_true, lon_true = tf.split(y_true_flat, num_or_size_splits=2, axis=1)
    lat_pred, lon_pred = tf.split(y_pred_flat, num_or_size_splits=2, axis=1)
    
    # Calculate the distance for each pair of points
    distances = haversine_distance(lat_true, lon_true, lat_pred, lon_pred)
    
    # Calculate the average distance
    average_distance = tf.reduce_mean(distances)
    
    return average_distance

In [6]:
input_steps = 60
num_features = 2
num_targets = 2
output_steps = 10

# Clear any previous models
tf.keras.backend.clear_session()

# Define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(input_steps, num_features)))
model.add(normalize_layer)
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dense(num_targets * output_steps))
model.add(tf.keras.layers.Reshape([output_steps, num_targets]))
model.add(denormalize_layer)

# Define optimizer
lr = 0.001
opt = tf.keras.optimizers.Adam(learning_rate=lr)

model.compile(loss='huber', optimizer=opt, metrics=['mae', 'mse', avg_m_diff, 'root_mean_squared_error'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 60, 2)          │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16)             │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │           340 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 10, 2)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_1 (Normalization) │ (None, 10, 2)          │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,566 (6.12 KB)

 Trainable params: 1,556 (6.08 KB)

 Non-trainable params: 10 (48.00 B)

In [11]:
model.load_weights("checkpoint/ckpt_16_lr001_v2.weights.h5")

c:\Users\adrie\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [12]:
## Define LR scheduling (optional if want to use or not)
start_lr = lr
min_lr = 0.00001
max_lr = 0.001
rampup_epochs = 0
sustain_epochs = 0
exp_decay = 0.0

# Define the scheduling function
def schedule(epoch):
  def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
    if epoch < rampup_epochs:
      lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
      lr = max_lr
    else:
      lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    return lr
  return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)

In [9]:
## Define callbacks and fit the model
log_dir = "logs/fit/n16_lr001_v2_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=200)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoint/ckpt_16_lr001_v2_1.weights.h5", 
    verbose=1, 
    save_weights_only=True,
    save_freq=1268*100)

model.fit(
  x=x_train, 
  y=y_train,
  epochs=2000,
  validation_data=(x_test, y_test),
  callbacks=[tensorboard_callback, cp_callback]) #Can add lr_scheduler

model.evaluate(x_test, y_test)

Epoch 1/5000
1268/1268 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - avg_m_diff: 5328.2104 - loss: 0.0017 - mae: 0.0297 - mse: 0.0035 - root_mean_squared_error: 0.0557 - val_avg_m_diff: 487.3859 - val_loss: 8.5404e-05 - val_mae: 0.0028 - val_mse: 1.7081e-04 - val_root_mean_squared_error: 0.0131
Epoch 2/5000
1268/1268 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - avg_m_diff: 346.3117 - loss: 3.2120e-05 - mae: 0.0020 - mse: 6.4241e-05 - root_mean_squared_error: 0.0078 - val_avg_m_diff: 205.4344 - val_loss: 1.4637e-05 - val_mae: 0.0012 - val_mse: 2.9273e-05 - val_root_mean_squared_error: 0.0054
Epoch 3/5000
1268/1268 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - avg_m_diff: 169.4029 - loss: 4.4196e-06 - mae: 9.7982e-04 - mse: 8.8391e-06 - root_mean_squared_error: 0.0030 - val_avg_m_diff: 140.9715 - val_loss: 5.1669e-06 - val_mae: 8.1433e-04 - val_mse: 1.0334e-05 - val_root_mean_squared_error: 0.0032
Epoch 4/5000
1268/1268 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - avg_m_diff: 125.6222 - loss: 1.7025e-06 - mae: 7.2262e-04

In [13]:
## Save the model
model.save('model_16_lr001_v2.keras')